In [1]:
import igvf_and_terra_api_tools as api_tools
import portal_to_terra_input_from_anaset as portal2terra_transfer
import terra_to_portal_posting as terra2portal_transfer

import firecloud.api as fapi

import importlib
import pandas as pd
import subprocess
import os

In [13]:
importlib.reload(portal2terra_transfer)

<module 'portal_to_terra_input_from_anaset' from '/Users/zheweishen/IGVF/IGVF_Repos/sc-pipeline-management/src/portal_to_terra_input_from_anaset.py'>

### Set up some params for pushing and posting

In [2]:
# Terra workspaces
terra_namespace = 'DACC_ANVIL'
terra_workspace = 'Playground for IGVF Single-Cell Data Processing'
test_run_terra_workspace = 'IGVF Single-Cell Data Processing'

# # Input analysis sets
igvf_sandbox_analysis_sets = ['TSTDS33660419', 'TSTDS12024147']
igvf_production_analysis_sets = ['IGVFDS5316CVFR', 'IGVFDS4768JVVU', 'IGVFDS8343PSUS']
igvf_production_analysis_sets_2 = ['IGVFDS9664YLAD', 'IGVFDS0223KLTB', 'IGVFDS0657NHTA']

# Terra etype for the pipeline input table
testrun_terra_etype = 'DACC_test_run'

# Where final barcode onlists are stored
local_final_barcode_dir = os.path.join(os.getcwd(), 'final_barcode_list')

# The GCP bucket the final barcode onlists will be stored
gs_final_barcode_dir = 'gs://fc-secure-de19fd29-2253-41cd-9751-1788cf7ad1a5/submissions/final_barcode_onlist/'

In [5]:
# Sandbox auth and APIs
igvf_api_keys_sandbox = api_tools.set_up_api_keys(igvf_endpoint='sandbox')
igvf_api_sandbox = api_tools.get_igvf_client_auth(igvf_site='sandbox',
                                                  igvf_api_keys=igvf_api_keys_sandbox)
iu_conn_sandbox = api_tools.get_igvf_utils_connection(igvf_utils_mode='sandbox',
                                                      igvf_api_keys=igvf_api_keys_sandbox,
                                                      submission_mode=True)

# Production auth and APIs
igvf_api_keys_production = api_tools.set_up_api_keys(igvf_endpoint='production')
igvf_api_production = api_tools.get_igvf_client_auth(igvf_site='production',
                                                     igvf_api_keys=igvf_api_keys_production)
iu_conn_production = api_tools.get_igvf_utils_connection(igvf_utils_mode='production',
                                                         igvf_api_keys=igvf_api_keys_production,
                                                         submission_mode=False)

2025-04-09 13:37:38,524:iu_debug:	submission=True: In submission mode.
2025-04-09 13:37:38,527:iu_debug:	submission=True: In submission mode.


In [3]:
# If FireCloud session somehow expires, usually when something updates on the platform
fapi._set_session()

### Generate the Terra data table from IGVF portal

#### Get the input parameter full table

In [6]:
# Construct the input table from portal based on analysis set accessions
portal_to_terra_input_table = portal2terra_transfer.generate_pipeline_input_table(query_analysis_set_accs=igvf_production_analysis_sets_2,
                                                                                  terra_etype=testrun_terra_etype,
                                                                                  local_barcode_file_dir=local_final_barcode_dir,
                                                                                  gs_barcode_list_bucket=gs_final_barcode_dir,
                                                                                  igvf_api=igvf_api_production
                                                                                  )

Processing: IGVFDS9664YLAD
Done: IGVFDS9664YLAD
Processing: IGVFDS0223KLTB
Done: IGVFDS0223KLTB
Processing: IGVFDS0657NHTA
Done: IGVFDS0657NHTA
Reformatting input table for Terra format...
Done.


In [7]:
# Take a peek
portal_to_terra_input_table

,analysis_set_acc,atac_MeaSetIDs,rna_MeaSetIDs,subpool_id,taxa,atac_read1_accessions,atac_read2_accessions,atac_barcode_accessions,rna_read1_accessions,rna_read2_accessions,...,atac_read_format,rna_barcode_inclusion_list,rna_read_format,onlist_mapping,possible_errors,chromap_index,genome_fasta,kb_index,genome_ref,kb_strand
entity:DACC_test_run_id,,,,,,,,,,,,,,,,,,,,,
IGVFDS9664YLAD,IGVFDS9664YLAD,[IGVFDS1013HCXI],[IGVFDS9139TBCB],IGVFSM0539NUPM,Homo sapiens,"[IGVFFI1187YZTV, IGVFFI4665EVGC]","[IGVFFI9371XVXW, IGVFFI4986VMDU]","[IGVFFI1676NVVR, IGVFFI0391NHGA]","[IGVFFI6948UZJO, IGVFFI4378SNUH]","[IGVFFI7951DAQB, IGVFFI5713MCUV]",...,"bc:8:23:-,r1:0:49,r2:0:49",gs://fc-secure-de19fd29-2253-41cd-9751-1788cf7...,"0,0,16:0,16,28:1,0,90",True,,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,gs://broad-buenrostro-pipeline-genome-annotati...,forward
IGVFDS0223KLTB,IGVFDS0223KLTB,[],[IGVFDS5272EQRG],IGVFSM8093RPKR,Mus musculus,[],[],[],"[IGVFFI9757ZTGD, IGVFFI4633WXZI]","[IGVFFI9526UYXA, IGVFFI9320MADV]",...,[],gs://fc-secure-de19fd29-2253-41cd-9751-1788cf7...,"1,10,18,1,48,56,1,78,86:1,0,10:0,0,140",False,,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,gs://broad-buenrostro-pipeline-genome-annotati...,forward
IGVFDS0657NHTA,IGVFDS0657NHTA,[IGVFDS8487QXIK],[IGVFDS6617HQDI],IGVFSM4419MRHA,Mus musculus,"[IGVFFI1918YZDJ, IGVFFI6103IWOF]","[IGVFFI4773YQEF, IGVFFI7241VYRQ]","[IGVFFI4773YQEF, IGVFFI7241VYRQ]","[IGVFFI0777CMJH, IGVFFI0768XBIK]","[IGVFFI7330UNCB, IGVFFI8781BOZU]",...,"bc:115:122,bc:153:160,bc:191:198,r1:0:99,r2:0:99",gs://fc-secure-de19fd29-2253-41cd-9751-1788cf7...,"1,115,123,1,153,161,1,191,199:1,0,10:0,0,100",False,,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,https://api.data.igvf.org/reference-files/IGVF...,gs://broad-buenrostro-pipeline-genome-annotati...,forward


In [8]:
# Checkout the columns
portal_to_terra_input_table.columns

Index(['analysis_set_acc', 'atac_MeaSetIDs', 'rna_MeaSetIDs', 'subpool_id',
       'taxa', 'atac_read1_accessions', 'atac_read2_accessions',
       'atac_barcode_accessions', 'rna_read1_accessions',
       'rna_read2_accessions', 'rna_barcode_accessions', 'atac_seqspec_urls',
       'rna_seqspec_urls', 'atac_read1', 'atac_read2', 'atac_barcode',
       'rna_read1', 'rna_read2', 'rna_barcode', 'atac_barcode_inclusion_list',
       'atac_read_format', 'rna_barcode_inclusion_list', 'rna_read_format',
       'onlist_mapping', 'possible_errors', 'chromap_index', 'genome_fasta',
       'kb_index', 'genome_ref', 'kb_strand'],
      dtype='object')

In [9]:
# Save the input file locally for inspection and manual upload to Terra
portal2terra_transfer.save_pipeline_input_table(pipeline_input_table=portal_to_terra_input_table,
                                                output_dir='./terra_datatable/'
                                               )

#### Optional: Upload to Terra directly if nothing to visually inspect

In [66]:
# Upload this to Terra under the name 'DACC_Tester'
api_tools.upload_portal_input_tsv_to_terra(terra_namespace=terra_namespace,
                                             terra_workspace=terra_workspace,
                                             terra_etype=testrun_terra_etype,
                                             portal_input_table=portal_to_terra_input_table,
                                             verbose=True
                                            )

<Response at 12af7b450: <Response [200]>> 
  _content: <str at 0x12b03f5f0>: "b'Pipeline_run_tester'"
  _content_consumed: True
  _next: None
  status_code: 200
  headers: <CaseInsensitiveDict at 12b2a8e90: {'Date': 'Fri, 21 Feb 2025 05:30:35 GMT', 'Server': 'Apache', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Cache-control': 'no-store', 'Pragma': 'no-cache', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'authorization,content-type,accept,origin,x-app-id', 'Access-Control-Allow-Methods': 'GET,POST,PUT,PATCH,DELETE,OPTIONS,HEAD', 'Access-Control-Max-Age': '1728000', 'Content-Type': 'text/plain; charset=UTF-8', 'Content-Length': '19', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}>
    object contents suppressed (instance from different module)
  r

In [19]:
terra_workspace

'Playground for IGVF Single-Cell Data Processing'

### POSTing pipeline output data to IGVF data portal (Only for recreating metadata objects if needed. The file uploads are best done in Terra VM)

#### Set up POST params and retrieve data table from Terra

In [ ]:
# For posting (this mock table mixes the input table because it has IGVF accessions and some output from Team 2 because I need files to try posting)
terra_to_portal_etype = 'DACC_mockTeam_2_tester_merged'
Sandbox_test_analysis_set = 'TSTDS33660419'

In [ ]:
# Generate the input data table for Terra
terra_to_portal_post_datatable = api_tools.get_terra_tsv_data_table(terra_namespace=terra_namespace,
                                                                    terra_workspace=terra_workspace,
                                                                    terra_etype=terra_to_portal_etype
                                                                   )
terra_to_portal_post_datatable.head(2)

#### Post all successful run results to the portal

In [ ]:
# Run all posting jobs
terra_to_portal_post_runs = terra2portal_transfer.post_all_successful_runs(igvf_api=igvf_api_sandbox, igvf_utils_api=iu_conn_sandbox, upload_file=False, full_terra_data_table=terra_to_portal_post_datatable)

In [ ]:
# Get the full summary
terra_to_portal_post_summary = terra2portal_transfer.summarize_post_status(post_results=terra_to_portal_post_runs)

In [ ]:
# Update the original output table with brief post summary
terra_to_portal_post_summary = terra2portal_transfer.add_post_status_summary_to_output_data_table(full_terra_data_table=terra_to_portal_post_datatable, post_status_df=terra_to_portal_post_summary)

#### Optional

In [ ]:
# Upload the run results to Terra
terra_to_portal_postres_etype = 'DACC_mockTeam_2_tester_merged'
api_tools.upload_output_post_res_to_terra(terra_namespace=terra_namespace,
                                          terra_workspace=terra_workspace,
                                          terra_etype=terra_to_portal_postres_etype,
                                          verbose=True
                                         )